<a href="https://colab.research.google.com/github/spalominor/SS202501/blob/main/SS_Taller05_140525.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Simulación manual de una cola M/G/1 para 50 clientes con tiempos entre llegadas
exponenciales (media = 4.5 minutos) y tiempos de servicio normales (media = 3.2,
desviación estándar = 0.6). Se calculan los tiempos de llegada, inicio y fin de
servicio, tiempo en cola y tiempo total en el sistema.

Librerías:
    numpy: para generar valores aleatorios.
    pandas: para organizar los datos en un DataFrame.
"""

# Importa la librería numpy para generación de números aleatorios
import numpy as np

# Importa la librería pandas para manejar la estructura tipo DataFrame
import pandas as pd

# Fija la semilla aleatoria para reproducibilidad
np.random.seed(42)

# Define el número de clientes a simular
num_clientes = 50

# Genera los tiempos entre llegadas (exponencial, media = 4.5 minutos)
# Cliente 1 llega en t = 0, por eso se antepone con 0.0
tiempos_entre_llegadas = [0.0] + list(
    np.round(np.random.exponential(scale=4.5, size=num_clientes - 1), 2)
)

# Calcula los tiempos absolutos de llegada como suma acumulada
tiempos_llegada = np.round(np.cumsum(tiempos_entre_llegadas), 2)

# Genera los tiempos de servicio (normal, media = 3.2, desviación estándar = 0.6)
# Se usa abs para evitar tiempos de servicio negativos
tiempos_servicio = np.round(
    np.abs(np.random.normal(loc=3.2, scale=0.6, size=num_clientes)), 2
)

# Inicializa listas para almacenar resultados intermedios
inicio_servicio = []
fin_servicio = []
tiempo_en_cola = []
tiempo_en_sistema = []

# Inicializa la variable de tiempo del servidor (cuando queda libre)
tiempo_servidor_libre = 0.0

# Itera por cada cliente para calcular su paso por el sistema
for i in range(num_clientes):
    # Calcula el inicio de servicio como el máximo entre llegada y liberación
    inicio = max(tiempos_llegada[i], tiempo_servidor_libre)
    inicio_servicio.append(round(inicio, 2))

    # Calcula el fin de servicio
    fin = inicio + tiempos_servicio[i]
    fin_servicio.append(round(fin, 2))

    # Calcula el tiempo en cola (espera)
    espera = inicio - tiempos_llegada[i]
    tiempo_en_cola.append(round(espera, 2))

    # Calcula el tiempo total en el sistema
    total_sistema = fin - tiempos_llegada[i]
    tiempo_en_sistema.append(round(total_sistema, 2))

    # Actualiza el tiempo en que el servidor quedará libre
    tiempo_servidor_libre = fin

# Construye el DataFrame con los resultados
df_simulacion = pd.DataFrame({
    'Cliente': np.arange(1, num_clientes + 1),
    'Tiempo entre llegadas': tiempos_entre_llegadas,
    'Tiempo de llegada': tiempos_llegada,
    'Tiempo de servicio': tiempos_servicio,
    'Inicio de servicio': inicio_servicio,
    'Fin de servicio': fin_servicio,
    'Tiempo en cola': tiempo_en_cola,
    'Tiempo en sistema': tiempo_en_sistema
})

# También puedes visualizar toda la tabla si lo deseas:
print(df_simulacion.to_string(index=False))


 Cliente  Tiempo entre llegadas  Tiempo de llegada  Tiempo de servicio  Inicio de servicio  Fin de servicio  Tiempo en cola  Tiempo en sistema
       1                   0.00               0.00                3.33                0.00             3.33            0.00               3.33
       2                   2.11               2.11                3.73                3.33             7.06            1.22               4.95
       3                  13.55              15.66                2.59               15.66            18.25            0.00               2.59
       4                   5.93              21.59                2.25               21.59            23.84            0.00               2.25
       5                   4.11              25.70                3.66               25.70            29.36            0.00               3.66
       6                   0.76              26.46                2.88               29.36            32.24            2.90               5.78

In [ ]:
# Importación del módulo random para generación de números pseudoaleatorios
import random
# Importación del módulo math para operaciones matemáticas
import math


# Fijar la semilla para que los resultados sean reproducibles
random.seed(1035972895)


def generar_exponencial(lambda_, n):
    """
    Genera una lista de valores pseudoaleatorios con distribución exponencial.

    Args:
        lambda_ (float): Parámetro de tasa (λ) de la distribución exponencial.
        n (int): Cantidad de valores pseudoaleatorios a generar.

    Returns:
        list: Lista de valores con distribución exponencial.
    """
    # Crear una lista vacía para almacenar los resultados
    resultados = []

    # Repetir n veces para generar n valores
    for _ in range(n):
        # Generar un número aleatorio uniforme en el intervalo (0, 1)
        u = random.random()

        # Aplicar la transformada inversa de la distribución exponencial
        x = -math.log(1 - u) / lambda_

        # Agregar el valor generado a la lista de resultados
        resultados.append(x)

    # Retornar la lista completa de valores
    return resultados


def generar_normal(mu, sigma, n):
    """
    Genera una lista de valores pseudoaleatorios con distribución normal,
    utilizando el método de Box-Muller.

    Args:
        mu (float): Media de la distribución normal.
        sigma (float): Desviación estándar de la distribución normal.
        n (int): Cantidad de valores pseudoaleatorios a generar.

    Returns:
        list: Lista de valores con distribución normal.
    """
    # Crear una lista vacía para almacenar los resultados
    resultados = []

    # Repetir n//2 veces porque Box-Muller genera 2 valores por iteración
    for _ in range(n // 2):
        # Generar dos números aleatorios uniformes en el intervalo (0, 1)
        u1 = random.random()
        u2 = random.random()

        # Aplicar la transformación Box-Muller para obtener z0 y z1
        z0 = math.sqrt(-2 * math.log(u1)) * math.cos(2 * math.pi * u2)
        z1 = math.sqrt(-2 * math.log(u1)) * math.sin(2 * math.pi * u2)

        # Escalar los valores z0 y z1 a la media y desviación deseadas
        x0 = z0 * sigma + mu
        x1 = z1 * sigma + mu

        # Agregar ambos valores a la lista de resultados
        resultados.append(x0)
        resultados.append(x1)

    # Si n es impar, generar un valor adicional
    if n % 2 == 1:
        # Generar dos números aleatorios uniformes
        u1 = random.random()
        u2 = random.random()

        # Solo usar z0, descartar z1
        z0 = math.sqrt(-2 * math.log(u1)) * math.cos(2 * math.pi * u2)

        # Escalar el valor y agregarlo a los resultados
        x0 = z0 * sigma + mu
        resultados.append(x0)

    # Retornar la lista de valores generados
    return resultados

# Definir el parámetro lambda
lambda_ = 1 / 4.5

# Definir la media y desviación estándar
mu = 3.2
sigma = 0.6

# Número de valores a generar
n = 50

tiempos_exponenciales = generar_exponencial(lambda_, n)
tiempos_normales = generar_normal(mu, sigma, n)

# Crear lista de tiempos acumulados de llegada
tiempos_llegada = []

# Inicializa el acumulador
acumulado = 0.0

# Recorre los tiempos entre llegadas
for t in tiempos_exponenciales:
    acumulado += t
    tiempos_llegada.append(acumulado)

# Mostrar tiempos de llegada y servicio
for llegada, servicio in zip(tiempos_llegada, tiempos_normales):
    print(f"{round(llegada, 2)}  --  {round(servicio, 2)}")


6.2  --  2.14
10.08  --  2.92
12.54  --  4.26
16.29  --  3.17
21.24  --  2.93
43.8  --  4.05
49.65  --  3.8
64.14  --  3.74
64.2  --  3.17
64.27  --  2.65
75.66  --  2.97
81.94  --  3.09
83.08  --  3.37
85.3  --  4.35
85.33  --  2.73
97.13  --  2.55
105.18  --  2.73
105.77  --  3.47
106.01  --  3.39
106.14  --  2.46
108.58  --  2.72
109.9  --  3.72
115.86  --  3.56
116.07  --  3.78
127.26  --  2.55
129.14  --  2.74
134.64  --  3.03
137.16  --  3.24
137.94  --  4.47
139.33  --  2.17
153.33  --  2.9
154.28  --  3.16
158.86  --  4.91
171.07  --  4.27
171.36  --  4.07
177.05  --  2.69
180.24  --  2.86
183.97  --  3.21
198.1  --  3.23
200.52  --  2.46
212.13  --  2.4
215.81  --  3.65
223.33  --  3.47
228.41  --  3.26
237.92  --  3.92
243.95  --  3.42
254.27  --  3.44
254.98  --  3.07
256.89  --  1.99
257.66  --  3.27
